In [ ]:
# Login to Hugging Face
from huggingface_hub import login

# Option 1: Will prompt for token
# login()

# Option 2: Use token directly (replace with your token)
# login(token="YOUR_HF_TOKEN_HERE")

# Option 3: Read from environment variable
# import os
# login(token=os.environ.get("HF_TOKEN"))


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-Math-7B-Instruct")

In [ ]:
# Load model directly

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-Math-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


What is your name?assed
What is your name?assed
What is your name?assed
What is your name?assed
What is your name?assed
What is your name?


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [3]:
from datasets import load_dataset

ds = load_dataset("qwedsacf/competition_math")

Generating train split: 100%|██████████| 12500/12500 [00:00<00:00, 414080.48 examples/s]


In [4]:
# Load the fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM

#finetuned_tokenizer = AutoTokenizer.from_pretrained("./qwen_finetuned")
finetuned_model = AutoModelForCausalLM.from_pretrained("./models/qwen2-math-7b-instruct_finetuned_on_first_3542_transformed_omni_math_solutions_filtered_lr:{lr}_warmup_steps:{warmup_steps}_num_epochs:{num_epochs}")
finetuned_model.to(device)


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./models/qwen2-math-7b-instruct_finetuned_on_first_3542_transformed_omni_math_solutions_filtered_lr:{lr}_warmup_steps:{warmup_steps}_num_epochs:{num_epochs}.

In [7]:
# Get a geometry level 5 problem from the dataset
geometry_level5 = ds['train'].filter(lambda x: x.get('type') == 'Geometry' and x.get('level') == 'Level 4')
problem = geometry_level5[2]


# Format the problem for the model
messages = [
    {"role": "user", "content": f"Solve this geometry problem without using any external tools. Put your solution in \\boxed{...} format.\n\n Here is the problem:\n\n{problem['problem']}"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(device)

inputs


Filter: 100%|██████████| 12500/12500 [00:00<00:00, 258118.64 examples/s]


{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,     50,   3948,    419,  17047,
           3491,   2041,   1667,    894,   9250,   7375,     13,  10224,    697,
           6291,    304,   1124,  79075,  42710,  47402,   3561,    382,   5692,
            374,    279,   3491,   1447,  11411,    400,     32,   4080,     19,
             11,     16,    701,    425,   4080,     16,     11,     19,  15087,
            323,    400,     34,   4080,     16,     11,     16,  15087,    525,
            279,  17228,    315,  57960,  55114,  19360,  12947,   3555,    686,
            387,    279,  13934,    315,    279,   2168,    315,   1459,    362,
            421,  57960,  55114,  19360,      3,    374,  45620,    220,     24,
             15,  12348,  65670,    911,    279,   6238,     30, 151645,    198,
         151644,  77091,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 

In [ ]:

outputs = finetuned_model.generate(**inputs, max_new_tokens=512)
response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print("Problem:")
print(problem['problem'])
print("\n" + "="*80 + "\n")
print("Model Response:")
print(response)

NameError: name 'model' is not defined

In [34]:
problem['solution']

'When we rotate images $90^{\\circ}$ the coordinates switch places, and the signs are adjusted based on whether or not an axis was crossed. In this case, rotating point $A$ $90^{\\circ}$ will bring it across the $y$-axis into Quadrant I, which means both the $x$ and $y$ will be positive. The original point $A$ was at $(-4, 1)$ so the final image will be at $(1, 4)$. We also could solve this problem by seeing that the slope of the segment from the origin to $A$ is $-1/4$. If $A$ is moving to a location that is a $90^{\\circ}$ rotation about the origin, it will move to a point on the segment perpendicular to the one that currently connects it to the origin. This will be the segment that has a slope of 4/1 or $-4/-1$ from the origin which puts us at $(1, 4)$ or $(-1, -4)$. The point $\\boxed{(1, 4)}$ is in the clockwise direction we need.'

In [26]:
import datasets
generated_dataset_level5 = datasets.load_from_disk('data/math_solutions_dataset_20000/')

In [ ]:
generated_dataset_level5

[0]

In [15]:
# Reload the dataset to ensure it's loaded correctly
from datasets import load_from_disk

generated_dataset_level5 = load_from_disk('data/math_solutions_dataset_20000/')

# Check the dataset structure
print(f"Dataset type: {type(generated_dataset_level5)}")
print(f"Dataset length: {len(generated_dataset_level5)}")
if hasattr(generated_dataset_level5, 'features'):
    print(f"Dataset features: {generated_dataset_level5.features}")
    print(f"\nFirst example:")
    print(generated_dataset_level5[0])
else:
    print(f"Error: Dataset is not a proper Dataset object. It's a {type(generated_dataset_level5)}")
    print(f"First item: {generated_dataset_level5[0] if len(generated_dataset_level5) > 0 else 'empty'}")


Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 553
Dataset features: {'problem': Value('string'), 'ground_truth': Value('string'), 'solution': Value('string')}

First example:
{'problem': 'Square ABCD has its center at $(8,-8)$ and has an area of 4 square units. The top side of the square is horizontal. The square is then dilated with the dilation center at (0,0) and a scale factor of 2. What are the coordinates of the vertex of the image of square ABCD that is farthest from the origin? Give your answer as an ordered pair.', 'ground_truth': 'With the center of dilation at the origin and a scale factor of 2, all the coordinates of square $ABCD$ are twice the coordinates of its preimage. The preimage has an area of 4 square units, so its side length is 2 units. Since the center of the preimage is at $(8, -8)$, the four vertices of the preimage are at $(7, -9), (7, -7), (9, -7)$ and $(9, -9)$. The point $(9, -9)$ is the farthest from the origin on the preimage, so 

In [ ]:
generated_dataset_level5[0]

{'problem': 'Square ABCD has its center at $(8,-8)$ and has an area of 4 square units. The top side of the square is horizontal. The square is then dilated with the dilation center at (0,0) and a scale factor of 2. What are the coordinates of the vertex of the image of square ABCD that is farthest from the origin? Give your answer as an ordered pair.',
 'ground_truth': 'With the center of dilation at the origin and a scale factor of 2, all the coordinates of square $ABCD$ are twice the coordinates of its preimage. The preimage has an area of 4 square units, so its side length is 2 units. Since the center of the preimage is at $(8, -8)$, the four vertices of the preimage are at $(7, -9), (7, -7), (9, -7)$ and $(9, -9)$. The point $(9, -9)$ is the farthest from the origin on the preimage, so the point farthest from the origin on the image of square $ABCD$ is $\\boxed{(18, -18)}.$',
 'solution': "\\begin{intermediatederivation}\nThe square has area \\(4\\), so its side length is \\(s=\\sq

In [ ]:
filtered_dataset = datasets.load_from_disk('newopenaioutputs/transformed_solutions_qwen2-math-7b-instruct_filtered')

NameError: name 'dataset' is not defined

In [1]:
filtered_dataset.num_rows

NameError: name 'filtered_dataset' is not defined